# Use case - truy vấn & phân tích dữ liệu

## Langchain agent

Ta có thể sử dụng `langchain` trong việc tự động hóa phân tích số liệu

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY', 'YourAPIKeyIfNotSet')

In [8]:
%%capture
!pip install langchain_experimental
!pip install langchain_community

In [28]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI
from langchain_experimental.sql import SQLDatabaseChain
from langchain_community.utilities.sql_database import SQLDatabase
llm = ChatOpenAI(temperature=0, model_name = 'gpt-3.5-turbo')

In [22]:
import os
import sqlite3
from langchain.agents import *
from langchain.sql_database import SQLDatabase
from langchain.chains import create_sql_query_chain
from langchain_community.llms import OpenAI
from langchain_openai import ChatOpenAI

Ta có thể truy vấn dữ liệu database

In [13]:
db_path = 'data/san_francisco_trees.db'
db = SQLDatabase.from_uri(f"sqlite:///{sqlite_db_path}")

In [12]:
# Read query
import sqlite3
import pandas as pd

In [15]:
# Kiểm tra các tables
conn = sqlite3.connect(db_path)

# Query to list all tables in the database
query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql_query(query, conn)

# Display the list of tables
print("Tables in the database:")
print(tables)

Tables in the database:
      name
0  SFTrees


In [18]:
query = """
SELECT * FROM SFTrees;
"""
df = pd.read_sql_query(query, conn)
df.head()

,TreeID,qSpecies,qAddress,PlantType
0,168225,Arbutus 'Marina' :: Hybrid Strawberry Tree,2547 Vallejo St,Tree
1,168228,Arbutus 'Marina' :: Hybrid Strawberry Tree,2547 Vallejo St,Tree
2,189160,Afrocarpus gracilior :: Fern Pine,100 Kissling St,Tree
3,189158,Afrocarpus gracilior :: Fern Pine,100 Kissling St,Tree
4,189161,Afrocarpus gracilior :: Fern Pine,100 Kissling St,Tree


Tạo chain với SQLDatabase

In [29]:
chain = create_sql_query_chain(llm, db)

In [31]:
chain.invoke({"question": "How many Species of trees are there in San Francisco?"})

"SELECT COUNT(DISTINCT qSpecies) AS NumSpecies\nFROM SFTrees\nWHERE qAddress LIKE '%San Francisco%'"

Với invoke, dữ liệu chưa thực sự thực hiện code

In [32]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)
agent_executor.run("How many Species of trees are there in San Francisco?")

/tmp/ipykernel_1182/3184071199.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  agent_executor.run("How many Species of trees are there in San Francisco?")




> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


SFTrees
Invoking: `sql_db_schema` with `{'table_names': 'SFTrees'}`



CREATE TABLE "SFTrees" (
	"TreeID" INTEGER, 
	"qSpecies" TEXT, 
	"qAddress" TEXT, 
	"PlantType" TEXT
)

/*
3 rows from SFTrees table:
TreeID	qSpecies	qAddress	PlantType
168225	Arbutus 'Marina' :: Hybrid Strawberry Tree	2547 Vallejo St	Tree
168228	Arbutus 'Marina' :: Hybrid Strawberry Tree	2547 Vallejo St	Tree
189160	Afrocarpus gracilior :: Fern Pine	100 Kissling St	Tree
*/
Invoking: `sql_db_query` with `{'query': 'SELECT DISTINCT qSpecies FROM SFTrees'}`


[("Arbutus 'Marina' :: Hybrid Strawberry Tree",), ('Afrocarpus gracilior :: Fern Pine',), ("Thuja occidentalis 'Emerald' :: Emerald Arborvitae",), ("Magnolia grandiflora 'Little Gem' :: Little Gem Magnolia",), ('Platanus x hispanica :: Sycamore: London Plane',), ('Ulmus parvifolia :: Chinese Elm',), ('Eriobotrya deflexa :: Bronze Loquat',), ('Leptospermum laevigatum :: Austral

'There are 310 different species of trees in San Francisco.'

Kiểm tra lại với pandas

In [36]:
df["qSpecies"].nunique()

578

## Pandas AI

In [1]:
%%capture
!pip install pandasai
!pip install pandasai[langchain]

In [7]:
%%capture
!pip install langchain_openai

In [10]:
from langchain_openai import OpenAI

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py)

In [11]:
pip show langchain

Name: langchain
Version: 0.1.20
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /opt/saturncloud/envs/saturn/lib/python3.11/site-packages
Requires: aiohttp, dataclasses-json, langchain-community, langchain-core, langchain-text-splitters, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [ ]:
langchain_llm = OpenAI(openai_api_key)

ImportError: cannot import name 'LangSmithParams' from 'langchain_core.language_models.chat_models' (/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/langchain_core/language_models/chat_models.py)

In [ ]:
df = SmartDataframe("data.csv", config={"llm": langchain_llm})

In [44]:
import os
import pandas as pd
from pandasai import Agent

employees_data = {
    'EmployeeID': [1, 2, 3, 4, 5],
    'Name': ['John', 'Emma', 'Liam', 'Olivia', 'William'],
    'Department': ['HR', 'Sales', 'IT', 'Marketing', 'Finance']
}

salaries_data = {
    'EmployeeID': [1, 2, 3, 4, 5],
    'Salary': [5000, 6000, 4500, 7000, 5500]
}

employees_df = pd.DataFrame(employees_data)
salaries_df = pd.DataFrame(salaries_data)

# By default, unless you choose a different LLM, it will use BambooLLM.
# You can get your free API key signing up at https://pandabi.ai (you can also configure it in your .env file)
os.environ["PANDASAI_API_KEY"] = os.getenv('PANDASAI_API_KEY')

In [45]:
agent = Agent([employees_df, salaries_df])
agent.chat("Who gets paid the most?")

Traceback (most recent call last):
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/pipelines/chat/generate_chat_pipeline.py", line 335, in run
    ).run(input)
      ^^^^^^^^^^
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/pipelines/pipeline.py", line 137, in run
    raise e
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/pipelines/pipeline.py", line 101, in run
    step_output = logic.execute(
                  ^^^^^^^^^^^^^^
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/pipelines/chat/prompt_generation.py", line 37, in execute
    self.logger.log(f"Using prompt: {prompt}")
                    ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/prompts/base.py", line 55, in __str__
    return self.to_string()
           ^^^^^^^^^^^^^^^^
  File "/opt/saturncloud/envs/saturn/lib/python3.11/site-packages/pandasai/prompts/base.py"

"Unfortunately, I was not able to answer your question, because of the following error:\n\n'Index' object has no attribute '_format_native_types'\n"